In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#hist_url = "https://drive.google.com/uc?export=download&id=1o66f5E4M0CYb9Xw9IVRnYxXbA_Jv09lP"
#sentiment_url = "https://drive.google.com/uc?export=download&id=1WHig3qMCOuTBEJncqe0IY6YIaOfO4Yss"

hist_url = "csv_files/historical_data.csv"
sentiment_url = "csv_files/fear_greed_index.csv"

trades = pd.read_csv(hist_url)
sentiment = pd.read_csv(sentiment_url)

In [ ]:
trades['time'] = pd.to_datetime(trades['Timestamp IST'], format='%d-%m-%Y %H.%M')
trades['date'] = trades['time'].dt.date

print(trades.isnull().sum())  
trades = trades.drop_duplicates()

In [ ]:
sentiment.head()

In [ ]:
sentiment['datetime'] = pd.to_datetime(sentiment['timestamp'], unit='s')
sentiment['date'] = sentiment['datetime'].dt.date

# Standardizing 'classification' to just 'Fear' or 'Greed'
def simplify_classification(x):
    if 'Fear' in x:
        return 'Fear'
    elif 'Greed' in x:
        return 'Greed'
    else:
        return 'Neutral'  

sentiment['Classification'] = sentiment['classification'].apply(simplify_classification)
sentiment = sentiment[['date', 'Classification', 'value']]  

In [ ]:
print(sentiment.head())
print(trades.head())

In [ ]:
merged = trades.merge(sentiment, left_on='date', right_on='date', how='left')
merged.head()

In [ ]:
merged = merged[merged['Classification'].notnull()]

In [ ]:
merged

### Closed PnL vs Sentiment

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Classification', y='Closed PnL', data=merged, palette='coolwarm')
plt.title('Profit/Loss Distribution Across Market Sentiment', fontsize=16)
plt.xlabel('Market Sentiment')
plt.ylabel('Closed PnL ($)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig('outputs/pnl_vs_sentiment.png')
plt.show()

### Trade Size (USD) vs Sentiment

In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(x='Classification', y='Size USD', data=merged, palette='Set2')
plt.title('Trade Size Distribution During Fear vs Greed')
plt.xlabel('Market Sentiment')
plt.ylabel('Trade Size ($)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig('outputs/trade_size_vs_sentiment.png')
plt.show() 

### Trade Side (Buy/Sell) Count Per Sentiment

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='Side', hue='Classification', data=merged, palette='Set2')
plt.title('Buy vs Sell Distribution During Fear vs Greed')
plt.xlabel('Trade Side')
plt.ylabel('Number of Trades')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig('outputs/side_sentiment_count.png')
plt.show()

### Top 10 Coins Traded by Sentiment

In [ ]:
top_coins = merged['Coin'].value_counts().head(10).index
coin_data = merged[merged['Coin'].isin(top_coins)]

plt.figure(figsize=(12,6))
sns.countplot(y='Coin', hue='Classification', data=coin_data, order=top_coins, palette='Spectral')
plt.title('Top 10 Coins Traded During Fear vs Greed')
plt.xlabel('Number of Trades')
plt.ylabel('Coin')
plt.tight_layout()
plt.savefig('outputs/top_coins_sentiment.png')
plt.show()

### Top 10 Trader Accounts: PnL vs Sentiment

In [ ]:
top_accounts = merged['Account'].value_counts().head(10).index
top_traders = merged[merged['Account'].isin(top_accounts)]

plt.figure(figsize=(14,6))
sns.boxplot(x='Account', y='Closed PnL', hue='Classification', data=top_traders, palette='Set1')
plt.title('Top Traders: PnL in Fear vs Greed')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('outputs/top_traders_pnl_sentiment.png')
plt.show()

### Line Plot of Fear-Greed Score Over Time

In [ ]:
plt.figure(figsize=(12,4))
sentiment_plot = merged[['date', 'value']].drop_duplicates().sort_values('date')

plt.plot(sentiment_plot['date'], sentiment_plot['value'], color='orange', linewidth=2)
plt.title('Fear-Greed Score Over Time', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Sentiment Score (0 = Extreme Fear, 100 = Extreme Greed)')
plt.grid(True)
plt.tight_layout()
plt.savefig('outputs/fear_greed_score_timeseries.png')
plt.show()